In [1]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image

# Load the trained model (ensure the path to the model is correct)
model = tf.keras.models.load_model(r"C:\Users\91636\Downloads\my_model.h5")  # Replace with your model's path

# Corrected Class Names (Ensure this matches the model's output order)
# Swap the order to fix the issue based on observed mismatches
class_names = ['Benign', 'Malignant', 'Non-Cancerous']

# Preprocessing Function
def preprocess_image(image):
    """
    Resize the image to 256x256 without scaling.
    Rescaling is already handled in the model.
    """
    image = image.resize((256, 256))  # Resize to match model input size
    image = np.array(image)  # Convert to NumPy array
    if len(image.shape) == 2:  # If grayscale, convert to RGB
        image = np.stack([image] * 3, axis=-1)
    return image  # Return image without scaling to [0, 1]

# Prediction Function
def predict(image):
    """
    Run predictions on a single preprocessed image.
    """
    # Add batch dimension to match model input shape
    input_array = np.expand_dims(image, axis=0)
    print("Input shape to model:", input_array.shape)  # Debugging

    # Get predictions
    probabilities = model.predict(input_array)
    print("Raw probabilities:", probabilities)  # Debugging

    # Get the predicted class and confidence
    predicted_class_idx = np.argmax(probabilities, axis=1)[0]
    confidence = probabilities[0][predicted_class_idx]

    # Return the corrected class name and confidence
    return class_names[predicted_class_idx], round(confidence * 100, 2)

# Gradio Interface Function
def gradio_interface(image):
    """
    Process the uploaded image and return the prediction result.
    """
    preprocessed_image = preprocess_image(image)  # Preprocess the image
    prediction, confidence = predict(preprocessed_image)  # Get predictions
    return f"{prediction} ({confidence}%)"  # Format the output for display

# Gradio Interface
with gr.Blocks(css="""
    body {
        background-color: #f5f5f5;
        font-family: Arial, sans-serif;
    }
    .gradio-container {
        background: linear-gradient(to bottom right, #ffffff, #e6e6e6);
        border-radius: 15px;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
        padding: 20px;
    }
    h1, .gr-markdown {
        color: #003366;
        text-align: center;
    }
    h1 {
        font-size: 2.5rem;
    }
    .gr-markdown {
        font-size: 1.2rem;
        margin-bottom: 20px;
    }
    .gr-button {
        background-color: #003366;
        color: white;
        border-radius: 5px;
        padding: 10px 20px;
    }
    .gr-button:hover {
        background-color: #00509e;
    }
""") as demo:
    gr.Markdown("""# Cancer Classification System""")
    gr.Markdown(
        """<p style='color: #003366;'>Upload an MRI image to classify it as Non-Cancerous, Benign, or Malignant. This system leverages a deep learning model trained on MRI data to provide accurate predictions.</p>"""
    )

    with gr.Row():
        image_input = gr.Image(type="pil", label="Upload MRI Image", image_mode="RGB")
        output_predictions = gr.Textbox(label="Predictions (Class, Confidence)")

    submit_button = gr.Button("Submit")

    submit_button.click(
        fn=gradio_interface,
        inputs=image_input,
        outputs=[output_predictions]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
